In [1]:
import numpy as np
import pandas as pd

In [3]:
train_data = pd.read_csv('/jupyter/Data/titanic/train.csv')

In [4]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Age, Cabin, Embarked 속성들이 missing value를 가지고 있다.  
Cabin, Name, Ticket 속성들은 무시한다.

In [6]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
train_data['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

Categorical 속성 조사

In [8]:
train_data['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [9]:
train_data['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [10]:
train_data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

Numeric 속성들을 처리하는 pipeline을 만든다.

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
    ('select_numeric', DataFrameSelector(['Age', 'SibSp', 'Parch', 'Fare'])),
    ('imputer', SimpleImputer(strategy='median'))
])

In [16]:
num_pipeline.fit_transform(train_data)

array([[22.    ,  1.    ,  0.    ,  7.25  ],
       [38.    ,  1.    ,  0.    , 71.2833],
       [26.    ,  0.    ,  0.    ,  7.925 ],
       ...,
       [28.    ,  1.    ,  2.    , 23.45  ],
       [26.    ,  0.    ,  0.    , 30.    ],
       [32.    ,  0.    ,  0.    ,  7.75  ]])

In [18]:
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X], index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

In [19]:
from sklearn.preprocessing import OneHotEncoder

In [20]:
cat_pipeline = Pipeline([
    ('select_cat', DataFrameSelector(['Pclass', 'Sex', 'Embarked'])),
    ('imputer', MostFrequentImputer()),
    ('cat_encoder', OneHotEncoder(sparse=False)),
])

In [25]:
cat_pipeline['cat_encoder'].categories_

[array([1, 2, 3], dtype=int64),
 array(['female', 'male'], dtype=object),
 array(['C', 'Q', 'S'], dtype=object)]

In [22]:
cat_pipeline.fit_transform(train_data)

array([[0., 0., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       ...,
       [0., 0., 1., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 1., 0.]])

In [26]:
cat_pipeline.fit_transform(train_data)[0]

array([0., 0., 1., 0., 1., 0., 0., 1.])

In [28]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Categorical, Numeric 속성을 통합

In [29]:
from sklearn.pipeline import FeatureUnion

preprocess_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline),
])

In [30]:
X_train = preprocess_pipeline.fit_transform(train_data)
X_train

array([[22.,  1.,  0., ...,  0.,  0.,  1.],
       [38.,  1.,  0., ...,  1.,  0.,  0.],
       [26.,  0.,  0., ...,  0.,  0.,  1.],
       ...,
       [28.,  1.,  2., ...,  0.,  0.,  1.],
       [26.,  0.,  0., ...,  1.,  0.,  0.],
       [32.,  0.,  0., ...,  0.,  1.,  0.]])

목표값 벡터

In [31]:
y_train = train_data['Survived']

In [33]:
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression(random_state=0).fit(X_train, y_train)

c:\users\함석현\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [34]:
a = np.c_[log_clf.decision_function(X_train), y_train, X_train]

In [37]:
df = pd.DataFrame(data=a, columns=['Score', 'Survived', 'Age', 'SibSp', 'Parch', 'Fare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Male', 'Female', 'Embarked_C', 'Embarked_Q', 'Embarked_S'])

In [38]:
df

,Score,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Male,Female,Embarked_C,Embarked_Q,Embarked_S
0,-2.333523,0.0,22.0,1.0,0.0,7.2500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,2.346187,1.0,38.0,1.0,0.0,71.2833,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.482929,1.0,26.0,0.0,0.0,7.9250,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,1.997224,1.0,35.0,1.0,0.0,53.1000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,-2.490937,0.0,35.0,0.0,0.0,8.0500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,-1.020188,0.0,27.0,0.0,0.0,13.0000,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
887,2.828927,1.0,19.0,0.0,0.0,30.0000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
888,-0.042273,0.0,28.0,1.0,2.0,23.4500,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
889,0.333530,1.0,26.0,0.0,0.0,30.0000,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


In [40]:
df.sort_values(by=['Score'], ascending=False)[:20]

,Score,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Male,Female,Embarked_C,Embarked_Q,Embarked_S
258,4.045890,1.0,35.0,0.0,0.0,512.3292,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
700,3.527883,1.0,18.0,1.0,0.0,227.5250,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
689,3.408115,1.0,15.0,0.0,1.0,211.3375,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
329,3.333936,1.0,16.0,0.0,1.0,57.9792,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
297,3.303696,0.0,2.0,1.0,2.0,151.5500,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
307,3.221292,1.0,17.0,1.0,0.0,108.9000,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
310,3.206062,1.0,24.0,0.0,0.0,83.1583,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
641,3.165982,1.0,24.0,0.0,0.0,69.3000,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
369,3.165982,1.0,24.0,0.0,0.0,69.3000,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
306,3.140283,1.0,28.0,0.0,0.0,110.8833,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0


In [41]:
df.sort_values(by=['Score'])[:20]

,Score,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Male,Female,Embarked_C,Embarked_Q,Embarked_S
159,-4.757342,0.0,28.0,8.0,2.0,69.5500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
324,-4.757342,0.0,28.0,8.0,2.0,69.5500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
201,-4.757342,0.0,28.0,8.0,2.0,69.5500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
846,-4.757342,0.0,28.0,8.0,2.0,69.5500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
851,-3.914882,0.0,74.0,0.0,0.0,7.7750,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
116,-3.455408,0.0,70.5,0.0,0.0,7.7500,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
326,-3.444945,0.0,61.0,0.0,0.0,6.2375,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
94,-3.369035,0.0,59.0,0.0,0.0,7.2500,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
683,-3.368008,0.0,14.0,5.0,2.0,46.9000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
13,-3.340547,0.0,39.0,1.0,5.0,31.2750,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
